<a href="https://colab.research.google.com/github/Rupneetk/DataViz/blob/master/HW_7_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import altair as alt
import folium
from folium import plugins
from folium import Marker
from altair import Chart
import pandas as pd

In [0]:
plants_data=pd.read_csv('Plants.csv')
plants_data.head()

,ID,State,Name,County,Lat,Lon,Combust,Fuel,FuelCat,Capacity,HeatInput,NetGen,NOX,SO2,CO2,CoalGen,OilGen,GasGen,NuclearGen,HydroGen,BiomassGen,WindGen,SolarGen,GeoGen,OtherFossilGen,OtherGen,NonRenewGen,RenewGen,CombGen,NonCombGen,CoalPortion,CapFac,SO2OutRate,CO2OutRate,NOXOutRate,NOXInRate,CO2InRate,SO2InRate
0,1,AK,Alakanuk,Wade Hampton,62.6833,-164.6544,1.0,JF,OIL,1.3,15801.5,1859.0,19.26,0.00,1262.7,0.0,1859.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1859.0,0.0,1859.0,0.0,0.0,0.163242,0.000000,0.679236,0.010360,0.001219,0.079910,0.000000
1,2,AK,Anchorage 1,Anchorage,61.2328,-149.8909,1.0,NG,GAS,103.1,666135.0,55428.0,92.00,1.32,39074.5,0.0,366.5,55061.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55428.0,0.0,55428.0,0.0,0.0,0.061371,0.000024,0.704960,0.001660,0.000138,0.058659,0.000002
2,3,AK,Angoon,Juneau,57.8425,-134.9772,1.0,DFO,OIL,1.5,16388.0,1928.0,19.98,0.66,1340.7,0.0,1928.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1928.0,0.0,1928.0,0.0,0.0,0.146728,0.000342,0.695384,0.010363,0.001219,0.081810,0.000040
3,4,AK,Aniak,Bethel,61.5875,-159.5317,1.0,DFO,OIL,1.1,28860.0,2725.0,35.18,1.19,2361.1,0.0,2725.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2725.0,0.0,2725.0,0.0,0.0,0.282794,0.000437,0.866459,0.012910,0.001219,0.081812,0.000041
4,5,AK,Annex Creek,Juneau,58.3167,-134.0958,0.0,WAT,HYDRO,4.0,0.0,26436.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,26436.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26436.0,0.0,26436.0,0.0,0.754452,0.000000,0.000000,0.000000,NaN,NaN,NaN


In [0]:
county_data=(pd.read_csv('County_code.csv'))
county_data.head()

,FIPS,Name
0,1001,Autauga
1,1003,Baldwin
2,1005,Barbour
3,1007,Bibb
4,1009,Blount


In [0]:
county_data=county_data.rename(columns={"Name":"County",'FIPS':'id'})
county_data.head()

,id,County
0,1001,Autauga
1,1003,Baldwin
2,1005,Barbour
3,1007,Bibb
4,1009,Blount


In [0]:
plants_data.columns

Index(['ID', 'State', 'Name', 'County', 'Lat', 'Lon', 'Combust', 'Fuel',
       'FuelCat', 'Capacity', 'HeatInput', 'NetGen', 'NOX', 'SO2', 'CO2',
       'CoalGen', 'OilGen', 'GasGen', 'NuclearGen', 'HydroGen', 'BiomassGen',
       'WindGen', 'SolarGen', 'GeoGen', 'OtherFossilGen', 'OtherGen',
       'NonRenewGen', 'RenewGen', 'CombGen', 'NonCombGen', 'CoalPortion',
       'CapFac', 'SO2OutRate', 'CO2OutRate', 'NOXOutRate', 'NOXInRate',
       'CO2InRate', 'SO2InRate'],
      dtype='object')

In [0]:
plants_data1=plants_data[['County','NonRenewGen', 'RenewGen','CO2','CoalGen']]
plants_data1.head()

,County,NonRenewGen,RenewGen,CO2,CoalGen
0,Wade Hampton,1859.0,0.0,1262.7,0.0
1,Anchorage,55428.0,0.0,39074.5,0.0
2,Juneau,1928.0,0.0,1340.7,0.0
3,Bethel,2725.0,0.0,2361.1,0.0
4,Juneau,0.0,26436.0,0.0,0.0


In [0]:
plants_data_CO2 = pd.pivot_table(plants_data1, index=['County'],values=['CO2'],aggfunc='sum')
plants_data_CO2.head()

,CO2
County,
ALAMOSA,0.0
Abbeville,99.5
Acadia,1379088.5
Accomack,48432.3
Ada,0.0


In [0]:
plants_data_CoalGen = pd.pivot_table(plants_data1, index=['County'],values=['CoalGen'],aggfunc='sum')
plants_data_CoalGen.head()

,CoalGen
County,
ALAMOSA,0.0
Abbeville,0.0
Acadia,0.0
Accomack,0.0
Ada,0.0


In [0]:
plants_data_NonRenewGen = pd.pivot_table(plants_data1, index=['County'],values=['NonRenewGen'],aggfunc='sum')
plants_data_NonRenewGen.head()

,NonRenewGen
County,
ALAMOSA,0.0
Abbeville,92.0
Acadia,3081100.0
Accomack,54412.9
Ada,0.0


In [0]:
plants_data_RenewGen = pd.pivot_table(plants_data1, index=['County'],values=['RenewGen'],aggfunc='sum')
plants_data_RenewGen.head()

,RenewGen
County,
ALAMOSA,8162.0
Abbeville,5293.0
Acadia,0.0
Accomack,0.0
Ada,435562.0


In [0]:
plant_data_merg= plants_data_CO2.merge(county_data, on='County')
plant_data_merg= plant_data_merg.merge(plants_data_NonRenewGen, on='County')
plant_data_merg= plant_data_merg.merge(plants_data_RenewGen, on='County')
plant_data_merg= plant_data_merg.merge(plants_data_CoalGen, on='County')
plant_data_merg.head()

,County,CO2,id,NonRenewGen,RenewGen,CoalGen
0,Abbeville,99.5,45001,92.0,5293.0,0.0
1,Acadia,1379088.5,22001,3081100.0,0.0,0.0
2,Accomack,48432.3,51001,54412.9,0.0,0.0
3,Ada,0.0,16001,0.0,435562.0,0.0
4,Adair,251.8,19001,268.0,497375.0,0.0


In [0]:
plant_data_merg["per_RenewGen"]=plant_data_merg.RenewGen/(plant_data_merg.NonRenewGen+plant_data_merg.RenewGen)*100
plant_data_merg.head()

,County,CO2,id,NonRenewGen,RenewGen,CoalGen,per_RenewGen
0,Abbeville,99.5,45001,92.0,5293.0,0.0,98.291551
1,Acadia,1379088.5,22001,3081100.0,0.0,0.0,0.000000
2,Accomack,48432.3,51001,54412.9,0.0,0.0,0.000000
3,Ada,0.0,16001,0.0,435562.0,0.0,100.000000
4,Adair,251.8,19001,268.0,497375.0,0.0,99.946146


In [0]:
from vega_datasets import data
counties=alt.topo_feature(data.us_10m.url, 'counties')

In [0]:
counties['url']

'https://vega.github.io/vega-datasets/data/us-10m.json'

In [0]:
alt.Chart(counties).mark_geoshape().encode(
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)

alt.Chart(...)

In [0]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [0]:
plot1= alt.Chart(counties).mark_geoshape().encode(
    color='CO2:Q',
    tooltip=['id:N', 'CO2:Q']
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(plant_data_merg, 'id', ['CO2'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)
plot2= alt.Chart(counties).mark_geoshape().encode(
    color='CoalGen:Q',
    tooltip=['id:N', 'CoalGen:Q']
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(plant_data_merg, 'id', ['CoalGen'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)
plot1 | plot2

alt.HConcatChart(...)

In [0]:
alt.Chart(counties).mark_geoshape().encode(
    color='per_RenewGen:Q',
    tooltip=['id:N', 'per_RenewGen:Q']
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(plant_data_merg, 'id', ['per_RenewGen'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)

alt.Chart(...)

State

In [0]:
State_data=(pd.read_csv('State.csv'))
State_data.head()

,State,id
0,AK,2
1,AL,1
2,AR,5
3,AS,60
4,AZ,4


In [0]:
plants_data2=plants_data[['State','NonRenewGen', 'RenewGen','CO2','CoalGen']]
plants_data.head()

,ID,State,Name,County,Lat,Lon,Combust,Fuel,FuelCat,Capacity,HeatInput,NetGen,NOX,SO2,CO2,CoalGen,OilGen,GasGen,NuclearGen,HydroGen,BiomassGen,WindGen,SolarGen,GeoGen,OtherFossilGen,OtherGen,NonRenewGen,RenewGen,CombGen,NonCombGen,CoalPortion,CapFac,SO2OutRate,CO2OutRate,NOXOutRate,NOXInRate,CO2InRate,SO2InRate
0,1,AK,Alakanuk,Wade Hampton,62.6833,-164.6544,1.0,JF,OIL,1.3,15801.5,1859.0,19.26,0.00,1262.7,0.0,1859.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1859.0,0.0,1859.0,0.0,0.0,0.163242,0.000000,0.679236,0.010360,0.001219,0.079910,0.000000
1,2,AK,Anchorage 1,Anchorage,61.2328,-149.8909,1.0,NG,GAS,103.1,666135.0,55428.0,92.00,1.32,39074.5,0.0,366.5,55061.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55428.0,0.0,55428.0,0.0,0.0,0.061371,0.000024,0.704960,0.001660,0.000138,0.058659,0.000002
2,3,AK,Angoon,Juneau,57.8425,-134.9772,1.0,DFO,OIL,1.5,16388.0,1928.0,19.98,0.66,1340.7,0.0,1928.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1928.0,0.0,1928.0,0.0,0.0,0.146728,0.000342,0.695384,0.010363,0.001219,0.081810,0.000040
3,4,AK,Aniak,Bethel,61.5875,-159.5317,1.0,DFO,OIL,1.1,28860.0,2725.0,35.18,1.19,2361.1,0.0,2725.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2725.0,0.0,2725.0,0.0,0.0,0.282794,0.000437,0.866459,0.012910,0.001219,0.081812,0.000041
4,5,AK,Annex Creek,Juneau,58.3167,-134.0958,0.0,WAT,HYDRO,4.0,0.0,26436.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,26436.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26436.0,0.0,26436.0,0.0,0.754452,0.000000,0.000000,0.000000,NaN,NaN,NaN


In [0]:
states = alt.topo_feature(data.us_10m.url, 'states')
states['url']

'https://vega.github.io/vega-datasets/data/us-10m.json'

In [0]:
alt.Chart(states).mark_geoshape().encode(
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)

alt.Chart(...)

In [0]:
plants_data2_NonRenewGen = pd.pivot_table(plants_data2, index=['State'],values=['NonRenewGen'],aggfunc='sum')
plants_data2_RenewGen = pd.pivot_table(plants_data2, index=['State'],values=['RenewGen'],aggfunc='sum')
plants_data2_CO2 = pd.pivot_table(plants_data2, index=['State'],values=['CO2'],aggfunc='sum')
plants_data2_CoalGen = pd.pivot_table(plants_data2, index=['State'],values=['CoalGen'],aggfunc='sum')
plant_data2_merg= plants_data2_CO2.merge(plants_data2_CoalGen, on='State')
plant_data2_merg= plant_data2_merg.merge(plants_data2_NonRenewGen, on='State')
plant_data2_merg= plant_data2_merg.merge(plants_data2_RenewGen, on='State')
plant_data2_merg= plant_data2_merg.merge(State_data, on='State')
plant_data2_merg.head()

,State,CO2,CoalGen,NonRenewGen,RenewGen,id
0,AK,3673740.9,620443.3,5308530.9,1440199.4,2
1,AL,86140283.9,63050410.7,140705591.6,11444920.6,1
2,AR,36350830.5,28152445.2,55716988.9,5283013.7,5
3,AZ,61330031.0,43643806.8,104586177.1,7164638.4,4
4,CA,52426224.3,2100221.3,145542605.0,59368312.6,6


In [0]:
plant_data2_merg["per_RenewGen"]=plant_data2_merg.RenewGen/(plant_data2_merg.NonRenewGen+plant_data2_merg.RenewGen)*100
plant_data2_merg.head()

,State,CO2,CoalGen,NonRenewGen,RenewGen,id,per_RenewGen
0,AK,3673740.9,620443.3,5308530.9,1440199.4,2,21.340302
1,AL,86140283.9,63050410.7,140705591.6,11444920.6,1,7.522105
2,AR,36350830.5,28152445.2,55716988.9,5283013.7,5,8.660678
3,AZ,61330031.0,43643806.8,104586177.1,7164638.4,4,6.411263
4,CA,52426224.3,2100221.3,145542605.0,59368312.6,6,28.972743


In [0]:
plot3=alt.Chart(states).mark_geoshape().encode(
    color='per_RenewGen:Q',
    tooltip=['id:N', 'per_RenewGen:Q']
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(plant_data2_merg, 'id', ['per_RenewGen'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300,
    title='Percentage of energy generation from renewables by State'
)
plot3

alt.Chart(...)

In [0]:
plant_data2_merg["per_Coal"]=plant_data2_merg.CoalGen/(plant_data2_merg.NonRenewGen+plant_data2_merg.RenewGen)*100
plant_data2_merg.head()

,State,CO2,CoalGen,NonRenewGen,RenewGen,id,per_RenewGen,per_Coal
0,AK,3673740.9,620443.3,5308530.9,1440199.4,2,21.340302,9.193482
1,AL,86140283.9,63050410.7,140705591.6,11444920.6,1,7.522105,41.439499
2,AR,36350830.5,28152445.2,55716988.9,5283013.7,5,8.660678,46.151548
3,AZ,61330031.0,43643806.8,104586177.1,7164638.4,4,6.411263,39.054576
4,CA,52426224.3,2100221.3,145542605.0,59368312.6,6,28.972743,1.024944


In [0]:
plot4=alt.Chart(states).mark_geoshape().encode(
    color='CO2:Q',
    tooltip=['id:N', 'CO2:Q']
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(plant_data2_merg, 'id', ['CO2'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)

plot5=alt.Chart(states).mark_geoshape().encode(
    color='per_Coal:Q',
    tooltip=['id:N', 'per_Coal:Q']
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(plant_data2_merg, 'id', ['per_Coal'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)

plot4 

alt.Chart(...)

  Second Type

In [0]:
CATEGORY= 'HYDRO' # GLOBAL VARIABLE --- UPPER case 
plants_data_subset=plants_data[plants_data.FuelCat==CATEGORY]
plants_data_subset.head()

,ID,State,Name,County,Lat,Lon,Combust,Fuel,FuelCat,Capacity,HeatInput,NetGen,NOX,SO2,CO2,CoalGen,OilGen,GasGen,NuclearGen,HydroGen,BiomassGen,WindGen,SolarGen,GeoGen,OtherFossilGen,OtherGen,NonRenewGen,RenewGen,CombGen,NonCombGen,CoalPortion,CapFac,SO2OutRate,CO2OutRate,NOXOutRate,NOXInRate,CO2InRate,SO2InRate
4,5,AK,Annex Creek,Juneau,58.3167,-134.0958,0.0,WAT,HYDRO,4.0,0.0,26436.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26436.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26436.0,0.0,26436.0,0.0,0.754452,0.0,0.0,0.0,NaN,NaN,NaN
8,9,AK,Beaver Falls,Ketchikan,55.3633,-131.6778,0.0,WAT,HYDRO,5.4,0.0,40956.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40956.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40956.0,0.0,40956.0,0.0,0.865804,0.0,0.0,0.0,NaN,NaN,NaN
12,13,AK,Black Bear Lake,Prince of Wales,55.4764,-133.1483,0.0,WAT,HYDRO,4.5,0.0,20822.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20822.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20822.0,0.0,20822.0,0.0,0.528209,0.0,0.0,0.0,NaN,NaN,NaN
13,14,AK,Blue Lake,Sitka,57.0516,-135.2297,0.0,WAT,HYDRO,7.0,0.0,58422.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58422.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58422.0,0.0,58422.0,0.0,0.952740,0.0,0.0,0.0,NaN,NaN,NaN
14,15,AK,Bradley Lake,Kenai Peninsula,59.7554,-150.8511,0.0,WAT,HYDRO,126.0,0.0,348271.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,348271.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,348271.0,0.0,348271.0,0.0,0.315531,0.0,0.0,0.0,NaN,NaN,NaN


In [0]:
plants_data_sub_lat_lng= plants_data_subset[['Lat', 'Lon']].as_matrix()
plants_data_sub_lat_lng

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([[  58.3167, -134.0958],
       [  55.3633, -131.6778],
       [  55.4764, -133.1483],
       ...,
       [  44.5161, -109.0922],
       [  42.9   , -110.9   ],
       [  42.7269, -110.8964]])

In [0]:
m= folium.Map([37.0902	, -95.7129], zoom_start=5)
m

In [0]:
m1=m.add_child(plugins.HeatMap(plants_data_sub_lat_lng))
m1